In [1]:
from CONSTANT import *
from tools import *
import pandas as pd
import numpy as np
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoderLayer, TransformerDecoder

from models import *

import torch

from torch import nn
import torch.nn.functional as F
from tqdm import tqdm


use_cuda = True
valid='loso'

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('using', device)

from representation.SigRepre import MultiSignalRepresentation

using cuda


In [2]:
class Params(object):
    def __init__(self, lr=0.001, epoch=100, valid='loso', target='valence', batch_size=64, use_cuda=use_cuda):
        self.batch_size = batch_size
        self.valid = valid
        self.target = target
        self.epoch = epoch
        self.lr = lr
        self.metrics_dict = {}
        self.debug = True
        self.out_channels = 32
        self.out_channels = 32
        self.hidden_size = 64
        self.num_layers = 1
        self.fcn_input = 50432
        self.dropout=0.2
        self.use_cuda=use_cuda
        self.device=torch.device(
            'cuda' if torch.cuda.is_available() and self.use_cuda else 'cpu')

In [3]:
spliter = load_model(r'./processed_signal/HKU956/400_4s_step_2s_spliter.pkl')
data = pd.read_pickle(r'./processed_signal/HKU956/400_4s_step_2s.pkl')
# data = pd.read_csv(r'./processed_signal/HKU956/400_4s_step_2s.csv')

for k in spliter[valid]:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [4]:
args = Params()
dataprepare = DataPrepare(args=args, target='valence', data=data, train_index=train_index, test_index=test_index, device=device, batch_size=4)
# train_dataloader, test_dataloader = dataprepare.get_data()

(1000, 4, 400) (1000, 1) (100, 4, 400) (100, 1)


In [5]:
xtest = dataprepare.xtest
xtrain = dataprepare.xtrain
ytest = dataprepare.ytest
ytrain = dataprepare.ytrain
xtest.shape, xtrain.shape, ytest.shape, ytrain.shape

(torch.Size([100, 4, 400]),
 torch.Size([1000, 4, 400]),
 torch.Size([100, 1]),
 torch.Size([1000, 1]))

In [12]:
model = MultiSignalRepresentation(output_size=40, device=args.device)
# model = model.to(args.device)
model.load_state_dict(torch.load(r'./output/0.0001_256_maskp0.8_checkpoint.pt'))

<All keys matched successfully>

In [13]:
from models import MER

In [14]:
model.output_layer = MER.MERClassifer(args, 2)

In [15]:
model = model.to(args.device)

In [16]:
model(xtest[:5]).shape

torch.Size([5, 2])

In [ ]:
# train_data, test_data = dataprepare.get_data()
# encoded_signals = []

# for i, (x, y) in tqdm(enumerate(train_data)):
#     encodedx = model.encoder(x).flatten(start_dim=1)
#     encoded_signals.append(encodedx)

In [ ]:
# encoded_xtrain = model.encoder(xtrain).flatten(start_dim=1)
# encoded_xtrain.shape